In [23]:
# Variable
W_max = 19.5;
W_min = 10;
import xlrd
import xlwt
import os


## excel parameter
# parameter schedule end Index
Schedule_endIndex = 289
# student schedule end Index
EndTime = 39;

# function for formating
def timeFormating(num):
    n = str(num)
    time = n.split(".")
    if time[1]=='5':
        time[1]='50' 
    min=str(int(time[1])*60/100).split(".")[0]
    if min =='0':
        min='00'
    if int(time[0]) <12:
        return time[0]+":"+min+"am"
    elif int(time[0])==12:
        return time[0]+":"+min+"pm"
    elif int(time[0])> 24:
        return str(int(time[0])-24)+":"+min+"am"
    elif int(time[0]) >12:
        return str(int(time[0])-12)+":"+min+"pm"
    
def dayFormating(num):
    if num==1:
        return "Mon"
    elif num ==2:
        return "Tue"
    elif num==3:
        return "Wed"
    elif num==4:
        return "Thu"
    elif num==5:
        return "Fri"
    elif num==6:
        return "Sat"
    elif num==7:
        return "Sun"

loc = ("schedule.xls")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
x = list(range(1, Schedule_endIndex))
#initial data
startTime = []
processStartTime = []
endTime = []
shiftslength = []
days = []
shiftstations = []
station = []

# get shift data
for i in x:
    timeStarted = sheet.cell_value(i,1)
    startTime.append(timeStarted)
    if str(timeStarted)[-3:] ==".25" or str(timeStarted)[-3:] ==".75" :
        processStartTime.append(timeStarted-0.25)
    else :
        processStartTime.append(timeStarted)  
    endTime.append(sheet.cell_value(i,2))
    shiftslength.append(sheet.cell_value(i,3))
    days.append(sheet.cell_value(i,4))
    shiftstations.append(sheet.cell_value(i,5))
    if sheet.cell_value(i,5) not in station:
        station.append(sheet.cell_value(i,5))

shifts = list(range(0, Schedule_endIndex-1))
# get all students availability and prefer time
studentName = []
p = []
#allAvailability[i]:availability for student i
#allAvailability[i][j]:availability for student i on day j
#allAvailability[i][j][k]:availability for student i on day j at time k
allAvailability = []
dirs = os.listdir("studentAvailability")
for file in dirs:
    if (not file.startswith("~$")) and (not file.startswith(".")) and (not file.startswith("_")):
        studentName.append(file.split(".")[0])
        # get each student availability
        wbp=xlrd.open_workbook("studentAvailability/"+file)
        sheetperson = wbp.sheet_by_index(0)
        # get prefer working time
        p.append(sheetperson.cell_value(1,8))
        time_slot = list(range(1, EndTime))
        stuAvailability = []
        for j in list(range(1,8)):
            day_slot = []
            for i in time_slot:
                day_slot.append(sheetperson.cell_value(i,j))
            stuAvailability.append(day_slot)
        allAvailability.append(stuAvailability)
# get whether shift is available for each student
a=[]
for i in range(len(studentName)):
    stuShiftAvailability = []
    stuAvailability = allAvailability[i]
    #loop all shifts
    for j in shifts:
        shiftstartTime = startTime[j]
        shiftLength = shiftslength[j]
        day = int(days[j]-1)
        startTimeSlot = int((shiftstartTime-7)/0.5)
        lengthSlot = int(shiftLength/0.5)
        # get student availability for the specific day
        stuDayAvailability = stuAvailability[day]
        # loop one single shift
        available = 1;
        for k in list(range(startTimeSlot,startTimeSlot+lengthSlot)):
            if(stuDayAvailability[k] ==0):
                available=0
                break
        stuShiftAvailability.append(available)
    a.append(stuShiftAvailability)

notavailable = []
for i in shifts:
    for j in range(i, len(shifts)):
        if days[i] == days[j] and i !=j:
            if startTime[i] <= startTime[j] and endTime[i] > startTime[j]:
                notavailable.append([i,j])
            elif endTime[j] > startTime[i] and endTime[j] <= endTime[i]:
                notavailable.append([i,j])

In [20]:
for i in notavailable:
    print(i)

[0, 1]
[0, 42]
[0, 43]
[0, 44]
[0, 109]
[0, 128]
[0, 129]
[0, 212]
[1, 2]
[1, 42]
[1, 43]
[1, 44]
[1, 109]
[1, 128]
[1, 129]
[1, 194]
[1, 212]
[1, 213]
[1, 255]
[2, 3]
[2, 43]
[2, 44]
[2, 45]
[2, 46]
[2, 110]
[2, 128]
[2, 129]
[2, 130]
[2, 131]
[2, 132]
[2, 133]
[2, 194]
[2, 195]
[2, 212]
[2, 213]
[2, 214]
[2, 255]
[2, 256]
[3, 44]
[3, 45]
[3, 46]
[3, 47]
[3, 110]
[3, 128]
[3, 129]
[3, 130]
[3, 131]
[3, 132]
[3, 133]
[3, 194]
[3, 195]
[3, 212]
[3, 213]
[3, 214]
[3, 255]
[3, 256]
[4, 5]
[4, 47]
[4, 48]
[4, 49]
[4, 50]
[4, 111]
[4, 133]
[4, 134]
[4, 135]
[4, 136]
[4, 137]
[4, 196]
[4, 197]
[4, 215]
[4, 217]
[4, 218]
[4, 257]
[4, 258]
[5, 47]
[5, 48]
[5, 49]
[5, 50]
[5, 111]
[5, 133]
[5, 134]
[5, 135]
[5, 136]
[5, 137]
[5, 196]
[5, 197]
[5, 215]
[5, 216]
[5, 217]
[5, 218]
[5, 257]
[5, 258]
[6, 7]
[6, 51]
[6, 52]
[6, 53]
[6, 112]
[6, 138]
[6, 139]
[6, 219]
[7, 51]
[7, 52]
[7, 53]
[7, 112]
[7, 138]
[7, 139]
[7, 219]
[7, 220]
[7, 259]
[8, 9]
[8, 53]
[8, 54]
[8, 55]
[8, 113]
[8, 138]
[8, 139]

[130, 212]
[130, 213]
[130, 214]
[130, 255]
[130, 256]
[131, 132]
[131, 133]
[131, 194]
[131, 195]
[131, 212]
[131, 213]
[131, 214]
[131, 255]
[131, 256]
[132, 133]
[132, 194]
[132, 195]
[132, 213]
[132, 214]
[132, 255]
[132, 256]
[133, 134]
[133, 135]
[133, 195]
[133, 196]
[133, 197]
[133, 214]
[133, 215]
[133, 216]
[133, 217]
[133, 218]
[133, 256]
[133, 257]
[133, 258]
[134, 135]
[134, 136]
[134, 196]
[134, 197]
[134, 215]
[134, 216]
[134, 217]
[134, 218]
[134, 257]
[134, 258]
[135, 136]
[135, 137]
[135, 196]
[135, 197]
[135, 215]
[135, 216]
[135, 217]
[135, 218]
[135, 257]
[135, 258]
[135, 273]
[135, 274]
[136, 137]
[136, 196]
[136, 197]
[136, 215]
[136, 216]
[136, 217]
[136, 218]
[136, 257]
[136, 258]
[136, 273]
[136, 274]
[137, 215]
[137, 257]
[138, 139]
[138, 198]
[138, 219]
[138, 220]
[138, 259]
[139, 140]
[139, 198]
[139, 199]
[139, 219]
[139, 220]
[139, 221]
[139, 259]
[139, 260]
[140, 141]
[140, 142]
[140, 198]
[140, 199]
[140, 219]
[140, 220]
[140, 221]
[140, 259]
[140, 260]

In [24]:
#del allAvailability
# model building
from gurobipy import *
import statistics
m = Model()
# initial variable
#x[i][j] assign student i to shift j
x=[]
for i in range(len(studentName)):
    y = [None]*len(shifts)
    x.append(y)
for i in range(len(studentName)):
    for j in shifts:
        x[i][j] = m.addVar(vtype=GRB.BINARY)
m.update()

# add constrain
for i in range(len(studentName)):
    expr = LinExpr()
    for j in shifts:
        if a[i][j] ==0:
            m.addConstr(x[i][j]==0)
        expr.add(x[i][j],shiftslength[j])
    m.addConstr(expr<=W_max)
    m.addConstr(expr>=W_min)

    for k in notavailable:
        m.addConstr(x[i][k[0]]+x[i][k[1]] <= 1)

for j in shifts:
    expr = LinExpr()
    for i in range(len(studentName)):
        expr.add(x[i][j],1)
    m.addConstr(expr<=1)

# goal
# F1
F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = []
    lateNightshift = 0
    for j in shifts:
        if x[i][j] == 1:
            scheduleTime += x[i][j]*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
            
            if shiftstations[j] == "Late Night":
                lateNightshift+=1
                
    F1 += (scheduleTime-p[i])*(scheduleTime-p[i])
    F2 += len(stuStation)-lateNightshift-len(unique)
# Define the objective function
ohm_1 = 2
ohm_2 = 10
m.Params.MIPGapAbs =0.00001
m.setObjective((ohm_1*F1+ ohm_2*F2)/len(studentName), GRB.MINIMIZE)
m.optimize()

Changed value of parameter MIPGapAbs to 1e-05
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Optimize a model with 122622 rows, 14400 columns and 279784 nonzeros
Model has 2080800 quadratic objective terms
Variable types: 0 continuous, 14400 integer (14400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 6e+00]
  QObjective range [2e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 2668.0750000
Presolve removed 120037 rows and 7884 columns
Presolve time: 0.85s
Presolved: 2585 rows, 6516 columns, 42522 nonzeros
Presolved model has 461462 quadratic objective terms
Variable types: 0 continuous, 6516 integer (6516 binary)

Root relaxation: objective 2.650000e+03, 1379 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2650.00000    0  106

In [27]:
workbook = xlwt.Workbook()
sheet = workbook.add_sheet('schedule')
sheet.write(0,0,"Shift")
sheet.write(0,1,"Time")
sheet.write(0,2,"Station")
for k, value in enumerate(shifts):
    sheet.write(int(k)+1, 0, k+1)
    sheet.write(int(k)+1, 1, dayFormating(days[k])+":"+timeFormating(startTime[k])+"-"+timeFormating(endTime[k]))
    sheet.write(int(k)+1, 2, shiftstations[k])
for j in range(0,len(studentName)):
    sheet.write(0,3+j,studentName[j])
    f = open("schedule/"+studentName[j]+".txt", "w")
    f.write(studentName[j]+":\n")
    writeMessage = ''
    totalHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalHour += x[j][i].x*shiftslength[i]
            sheet.write(i+1,3+j,"Assign")
            writeMessage += dayFormating(days[i])+": "+timeFormating(startTime[i])+"-"+\
                            timeFormating(endTime[i])+", ("+str(shiftstations[i])+")\n"
    f.write('Total hour: '+str(totalHour)+'\n')
    f.write(writeMessage)
    f.close()
workbook.save('assignSchedule.xls')
#check objective function 1
print("Result for objective function 1")
difference = 0
print("Student: Total time:\tPrefer:\tDifference")
for j in range(0,len(studentName)):
    writeMessage = ''
    totalstuHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalstuHour += x[j][i].x*shiftslength[i]
    difference+=abs(totalstuHour-p[j])
    print(studentName[j]+"\t "+str(totalstuHour)+"\t\t"+str(p[j])+"\t"+str(abs(totalstuHour-p[j])))
print("Total difference is "+str(difference))
print()
print()
# check objective function 2
print("Result for objective function 2")
totalStation = 0
totalunique = 0
print("Student: No. shifts:\tNo. different station:\tDifference")
for i in range(len(studentName)):
    stuStation = []
    unique = []
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    totalStation+=len(stuStation)
    totalunique+=len(unique)
    print(studentName[i]+"\t "+str(len(stuStation))+"\t\t"+str(len(unique))+"\t\t\t"+str(len(stuStation)- len(unique)))
print("Total\t "+str(totalStation)+"\t\t"+str(totalunique)+"\t\t\t"+str(totalStation-totalunique))

F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = []
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    F1 += (scheduleTime-p[i])*(scheduleTime-p[i])
    F2 += len(stuStation)-len(unique)
print()
print(F1)
print(F2)
ohm_1 = 2
ohm_2 = 5
print((ohm_1*F1+ ohm_2*F2)/len(studentName))

Result for objective function 1
Student: Total time:	Prefer:	Difference
11	 13.25		13.0	0.25
39	 15.0		15.5	0.5
38	 14.0		14.0	0.0
10	 14.75		15.0	0.25
12	 14.25		15.0	0.75
13	 10.25		10.0	0.25
17	 12.5		12.0	0.5
16	 13.5		14.0	0.5
28	 12.0		12.0	0.0
14	 13.75		13.5	0.25
15	 16.25		16.0	0.25
29	 14.25		15.0	0.75
8	 11.5		12.0	0.5
9	 17.0		17.0	0.0
48	 10.5		10.5	0.0
49	 14.0		13.5	0.5
1	 10.0		10.0	0.0
47	 10.25		10.0	0.25
46	 14.75		14.5	0.25
44	 11.25		11.5	0.25
50	 10.25		10.0	0.25
2	 13.5		13.5	0.0
3	 13.0		13.0	0.0
45	 12.5		12.5	0.0
41	 10.0		10.0	0.0
7	 14.0		14.0	0.0
6	 13.5		14.0	0.5
40	 13.0		13.0	0.0
4	 11.0		11.0	0.0
42	 11.0		11.0	0.0
43	 13.75		13.5	0.25
5	 12.5		12.5	0.0
30	 14.75		14.0	0.75
24	 11.5		12.0	0.5
18	 12.25		12.0	0.25
19	 12.5		12.5	0.0
25	 10.25		11.0	0.75
31	 13.5		13.5	0.0
27	 14.75		15.0	0.25
33	 13.0		13.0	0.0
32	 15.0		14.5	0.5
26	 13.5		13.5	0.0
22	 16.0		16.0	0.0
36	 12.0		12.0	0.0
37	 16.75		17.0	0.25
23	 16.75		17.0	0.25
35	 14.75		14.0	0.75
21	 14

In [28]:
# print remaining shifts
print("Shift\tStart Time\tEnd Time\tLength\tDay\tStation")
n = 0
for j in shifts:
    assign = 0
    for i in range(len(studentName)):
        if x[i][j].x==1:
            assign=1
            break
    if assign == 0:
        n+=1
        print(j,"\t",startTime[j],"\t\t",endTime[j],"\t\t",shiftslength[j],"\t",days[j],"\t",shiftstations[j])
print("Total number of unassigned shifts: ",n)

Shift	Start Time	End Time	Length	Day	Station
5 	 17.0 		 20.5 		 3.5 	 1.0 	 Big Kitchen
7 	 7.5 		 10.75 		 3.25 	 2.0 	 Big Kitchen
8 	 10.75 		 15.0 		 4.25 	 2.0 	 Big Kitchen
11 	 17.0 		 20.5 		 3.5 	 2.0 	 Big Kitchen
18 	 7.0 		 10.5 		 3.5 	 4.0 	 Big Kitchen
19 	 7.5 		 11.0 		 3.5 	 4.0 	 Big Kitchen
23 	 17.0 		 20.5 		 3.5 	 4.0 	 Big Kitchen
26 	 10.75 		 15.0 		 4.25 	 5.0 	 Big Kitchen
30 	 9.5 		 13.25 		 3.75 	 6.0 	 Big Kitchen
31 	 9.75 		 14.0 		 4.25 	 6.0 	 Big Kitchen
32 	 10.25 		 14.25 		 4.0 	 6.0 	 Big Kitchen
33 	 10.75 		 14.5 		 3.75 	 6.0 	 Big Kitchen
35 	 17.0 		 20.5 		 3.5 	 7.0 	 Big Kitchen
36 	 9.5 		 13.25 		 3.75 	 7.0 	 Big Kitchen
38 	 10.25 		 14.25 		 4.0 	 7.0 	 Big Kitchen
39 	 10.75 		 14.5 		 3.75 	 7.0 	 Big Kitchen
44 	 10.0 		 13.0 		 3.0 	 1.0 	 Deli
55 	 11.75 		 15.0 		 3.25 	 2.0 	 Deli
60 	 7.0 		 10.75 		 3.75 	 3.0 	 Deli
67 	 17.0 		 20.5 		 3.5 	 3.0 	 Deli
69 	 7.0 		 10.0 		 3.0 	 4.0 	 Deli
75 	 16.75 		 20.5 		 3.75 	 4.0